# Test de l'évaluation du modèle générique de NER de la chaîne de traitement en langue française de Stanza avec NERVAL

## Processus d'évaluation

On teste la librairie python [NERVAL](https://gitlab.com/teklia/nerval/-/tree/master) pour évaluer la prédiction du modèle générique de NER de la chaîne de traitement en langue française de [Stanza](https://stanfordnlp.github.io/stanza/ner.html).

Voir également : https://teklia.com/blog/202104-nerval/

Selon https://teklia.com/blog/202104-nerval/ :

"The automatic transcription is first aligned with the ground truth at character level, by minimising the Levenshtein distance between them. Each entity in the ground truth is then matched with an overlapping entity in the aligned transcription with the same entity label, or an empty character string if no match is found. If the edit distance between the two entities is less than 30% of the ground truth entity length, the predicted entity is considered as recognised. For the purpose of matching detected entities to existing databases, we estimated that a 70% match between the entity texts was a fair threshold."

Pour récapituler et traduire, NERVAL effectue donc les opérations suivantes pour évaluer :

1. Alignement de la transcription automatique avec la vérité de terrain au niveau du caractère en minimisant le distance de Levenshtein
2. Chaque entité annotée dans la vérité de terrain est associé à une entité possédant le même label et au même niveau dans le texte, dans la transcription alignée, ou avec un caractère vide si aucune association n'est trouvée. 
3. Une entité est considérée comme reconnue dans la transcription si la distance d'édition avec la vérité de terrain est moins de 30%.

## Données pour l'évaluation

La vérité de terrain NER résulte de l'annotation d'une page de répertoire transcrite manuellement sur la plateforme d'annotation Inception, de l'export des annotations au format CONLL 2002, et de la conversion du format CONLL 2002 au format d'annotation BIOES grâce à un script présent sur le repository Github [NCRFpp](https://github.com/jiesutd/NCRFpp). Voir ici : https://github.com/jiesutd/NCRFpp/blob/master/utils/tagSchemeConverter.py#L16

Le fichier texte FRAN\_0025\_5038\_L-0\_htr\_cer\_13_bioes.txt résulte de l'annotation NER réalisée avec la librairie python de NLP [Stanza](https://stanfordnlp.github.io/stanza/ner.html).

## Evaluation avec NERVAL

NERVAL permet de manipuler un seuil pour la distance d'édition. Par défaut, et comme mentionné plus haut, celui-ci est de 30%. Cette valeur est celle conseillé par l'équipe de développement de NERVAL. On commence par voir le résultat avec cette valeur par défaut : 

In [2]:
!nerval -a data/FRAN_0025_5038_L-0_gt.bioes -p data/FRAN_0025_5038_L-0_htr_cer_13.bioes

| tag  | predicted | matched | Precision | Recall | F1    | Support |
|------|-----------|---------|-----------|--------|-------|---------|
| PER  | 43        | 23      | 0.535     | 0.657  | 0.59  | 35      |
| ORG  | 2         | 0       | 0.0       | 0.0    | 0     | 4       |
| MISC | 2         | 0       | 0.0       | None   | None  | None    |
| LOC  | 32        | 17      | 0.531     | 0.486  | 0.507 | 35      |
| All  | 79        | 40      | 0.506     | 0.541  | 0.523 | 74      |


Les scores encouragent au fine-tuning d'un modèle de NER pour atteindre des scores proche des 90% pour chaque entité.

On descend le seuil à 20% :

In [4]:
!nerval -a data/FRAN_0025_5038_L-0_gt.bioes -p data/FRAN_0025_5038_L-0_htr_cer_13.bioes -t 0.20

| tag  | predicted | matched | Precision | Recall | F1    | Support |
|------|-----------|---------|-----------|--------|-------|---------|
| PER  | 43        | 20      | 0.465     | 0.571  | 0.513 | 35      |
| ORG  | 2         | 0       | 0.0       | 0.0    | 0     | 4       |
| MISC | 2         | 0       | 0.0       | None   | None  | None    |
| LOC  | 32        | 16      | 0.5       | 0.457  | 0.478 | 35      |
| All  | 79        | 36      | 0.456     | 0.486  | 0.471 | 74      |


Sans surprise, les scores sont moins bons.

On monte le seuil à 40% :

In [5]:
!nerval -a data/FRAN_0025_5038_L-0_gt.bioes -p data/FRAN_0025_5038_L-0_htr_cer_13.bioes -t 0.40

| tag  | predicted | matched | Precision | Recall | F1    | Support |
|------|-----------|---------|-----------|--------|-------|---------|
| PER  | 43        | 25      | 0.581     | 0.714  | 0.641 | 35      |
| ORG  | 2         | 0       | 0.0       | 0.0    | 0     | 4       |
| MISC | 2         | 0       | 0.0       | None   | None  | None    |
| LOC  | 32        | 17      | 0.531     | 0.486  | 0.507 | 35      |
| All  | 79        | 42      | 0.532     | 0.568  | 0.549 | 74      |


Avec plus de souplesse dans la gestion de la distance d'édition, les scores montent. 

En imposant un seuil d'édition à 0, et donc une correspondance exacte entre la vérité de terrain et la transcription, les scores sont naturellement très bas :

In [7]:
!nerval -a data/FRAN_0025_5038_L-0_gt.bioes -p data/FRAN_0025_5038_L-0_htr_cer_13.bioes -t 0

| tag  | predicted | matched | Precision | Recall | F1    | Support |
|------|-----------|---------|-----------|--------|-------|---------|
| PER  | 43        | 8       | 0.186     | 0.229  | 0.205 | 35      |
| ORG  | 2         | 0       | 0.0       | 0.0    | 0     | 4       |
| MISC | 2         | 0       | 0.0       | None   | None  | None    |
| LOC  | 32        | 15      | 0.469     | 0.429  | 0.448 | 35      |
| All  | 79        | 23      | 0.291     | 0.311  | 0.301 | 74      |
